In [24]:
import pandas as pd
import random

Descargamos los datos del callejero de Madrid desde aquí: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=b3c41f3cf6a6c410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [51]:
df_callejero = pd.read_csv('./data/VialesVigentes_20241226.csv', sep = ';', encoding = 'cp1252')

In [52]:
df_callejero.head(10)

,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,VIA_NOMBRE_ACENTOS,COD_VIA_COMIENZA,CLASE_COMIENZA,PARTICULA_COMIENZA,NOMBRE_COMIENZA,NOMBRE_ACENTOS_COMIENZA,COD_VIA_TERMINA,CLASE_TERMINA,PARTICULA_TERMINA,NOMBRE_TERMINA,NOMBRE_ACENTOS_TERMINA
0,31001337,AUTOVÍA,NaN,A-1,A-1,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
1,31001336,AUTOVÍA,NaN,A-2,A-2,310200,CALLE,DE,FRANCISCO SILVELA,FRANCISCO SILVELA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
2,31001342,AUTOVÍA,NaN,A-3,A-3,480800,PLAZA,DE,MARIANO DE CAVIA,MARIANO DE CAVIA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
3,31001334,AUTOVÍA,NaN,A-4,A-4,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
4,31001341,AUTOVÍA,NaN,A-42,A-42,468400,AVENIDA,DEL,MANZANARES,MANZANARES,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
5,31001340,AUTOVÍA,NaN,A-5,A-5,687520,GLORIETA,DE,SAN VICENTE,SAN VICENTE,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
6,31001339,AUTOVÍA,NaN,A-6,A-6,614700,GLORIETA,DE LA,PUERTA DE HIERRO,PUERTA DE HIERRO,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
7,31000210,CALLE,DEL,ABAD JUAN CATALAN,ABAD JUAN CATALÁN,978785,CARRETERA,DE,VICALVARO A LA ESTACION DE O'DONNELL,VICÁLVARO A LA ESTACIÓN DE O'DONNELL,99000001,LUGAR,NaN,FONDO DE SACO,FONDO DE SACO
8,200,CALLE,DE LA,ABADA,ABADA,656500,CALLE,DE LA,SALUD,SALUD,342950,CALLE,NaN,GRAN VIA,GRAN VÍA
9,300,CALLE,DE LOS,ABADES,ABADES,505000,CALLE,DEL,MESON DE PAREDES,MESÓN DE PAREDES,262000,CALLE,DE,EMBAJADORES,EMBAJADORES


In [53]:
df_calles = df_callejero[df_callejero.VIA_CLASE != 'AUTOVÍA'][['VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE']]

df_calles['NOMBRE_VIA'] = df_calles['VIA_PAR'] + ' ' + df_calles['VIA_NOMBRE']
df_calles.drop(columns = ['VIA_PAR', 'VIA_NOMBRE'], inplace  = True)


df_calles.rename(columns={'VIA_CLASE': 'TIPO_VIA'}, inplace = True)
df_calles.dropna(inplace =  True)

In [ ]:
df_callejero.head(10)

In [74]:
def generar_datos_entrenamiento(df, n):
    """
    Genera N líneas aleatorias a partir de un DataFrame base.
    
    Args:
        df (pd.DataFrame): DataFrame base para seleccionar líneas.
        n (int): Número de líneas aleatorias a generar.
    
    Returns:
        pd.DataFrame: DataFrame con N líneas aleatorias generadas.
    """
    lineas = []
    for _ in range(n):
        # Seleccionar una línea aleatoria del DataFrame base
        row = df.sample(1).copy()

        # Agregar un número aleatorio
        pre_numero = random.choice(['n*', 'nr', 'nº', '', 'n'])
        numero = random.randint(1, 50)
        post_numero = random.choice(['', '', '', ','])
        row['NUMERO'] = f'{pre_numero}{numero}{post_numero}'

        # Agregar combinación aleatoria de piso y letra
        piso = random.randint(1, 10)
        letra = random.choice(['A', 'B', 'C', 'D', 'IZDA', 'DCHA', 'CTRO'])
        espacio = random.choice([' ', '*', 'º', '', '-'])
        combinacion = f"{piso}{espacio}{letra}"
        row['RESTO'] = combinacion

        # Agregar la línea generada a la lista
        lineas.append(row)

    # Combinar todas las líneas en un nuevo DataFrame
    return pd.concat(lineas, ignore_index=True)


In [75]:
def crear_entidades(row):
    texto = row['texto']
    entidades = []
    start = 0
    for col, label in zip(['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO'], ['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO']):
        value = str(row[col])
        start = texto.find(value, start)  # Buscar el índice inicial de la entidad
        if start != -1:
            end = start + len(value)  # Índice final de la entidad
            entidades.append((start, end, label))
            start = end  # Actualizar el inicio para evitar errores en textos repetidos
    return (texto, {'entities': entidades})

In [76]:
df = generar_datos_entrenamiento(df_calles, 3000)
df['texto'] = df['TIPO_VIA'] + ' ' + df['NOMBRE_VIA'] + ' ' + df['NUMERO'].astype(str) + ' ' + df['RESTO']
df['entidades'] = df.apply(crear_entidades, axis=1)

In [77]:
df.to_csv('./data/train_set.csv', sep = ';', encoding='utf-8-sig', index = None)